## Import Packages

In [1]:
import pandas as pd
import numpy as np

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.utils import make_grid

import math
import random
import json

from PIL import Image, ImageOps, ImageEnhance
import numbers

import torchattacks
from torchattacks import CW

import matplotlib.pyplot as plt
%matplotlib inline

## Normalize Dataset

In [2]:
#Creating dataset using torch dataloaders
batch_size_train = 64
batch_size_test = 1000
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5,), (0.5,))]) # Normalizing dataset

# Training dataset
train_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=True, download=True,
                             transform=transform),
  batch_size=batch_size_train, shuffle=True)

# Test dataset
test_loader = torch.utils.data.DataLoader(
  torchvision.datasets.MNIST('/files/', train=False, download=True,
                             transform=transform),
  batch_size=batch_size_test, shuffle=True)

# Initialize GPU
use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")

## Initial Model Structure

In [3]:
# Initial model structure
class HNet(nn.Module):    
    def __init__(self):
        super(HNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 64)  # 6*6 from image dimension
#         self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
#         x = F.softmax(x, dim = 1)
        return x     

## SubModel Structure

In [4]:
# Submodel Structure for training residual
class NHNet(nn.Module):    
    def __init__(self):
        super(NHNet, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)


    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x, dim = 1) # add this because I need one-hot label and MSE loss
        return x   

## Initialize Data Structures

In [5]:
# Initialize some data structures to store useful data
train_losses = []
train_counter = []
test_losses = []

## Initialize Parameters, optimizer and loss function

In [6]:
# Create the initial model
initial_model = HNet()

#Create the optimizer for the initial model
optimizer = optim.Adam(initial_model.parameters(), lr=0.003)

# Create Loss function for the intial model
criterion = nn.CrossEntropyLoss()

# exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

#Change model into cuda mode
if torch.cuda.is_available():
    initial_model = initial_model.cuda()
    criterion = criterion.cuda()

## Training Function for Initial Model

In [7]:
# Initial Model Training Function
def train(epoch):
    initial_model.train()
#     exp_lr_scheduler.step()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = initial_model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
        train_counter.append(
                (batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
        torch.save(initial_model.state_dict(), 'C:/Users/cozyn/Desktop/Research/Adversarial-Machine-Learning/results/model.pth')
        torch.save(optimizer.state_dict(), 'C:/Users/cozyn/Desktop/Research/Adversarial-Machine-Learning/results/optimizer.pth')
        if (batch_idx + 1)% 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss.item()))

## Evaluation Function for Intial Model

In [8]:
#Initial Model Evaluating Function
def evaluate(data_loader):
    initial_model.eval()
    loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in data_loader:
            data, target = Variable(data, volatile=True), Variable(target)
            if torch.cuda.is_available():
                data = data.cuda()
                target = target.cuda()
        
            output = initial_model(data)
        
            loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
    test_losses.append(loss)    
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

## Start Training the Initial Model

In [9]:
# Start Training
n_epochs = 65

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

Train Epoch: 0 [6400/60000 (11%)]	Loss: 0.392823
Train Epoch: 0 [12800/60000 (21%)]	Loss: 0.570041
Train Epoch: 0 [19200/60000 (32%)]	Loss: 0.565410
Train Epoch: 0 [25600/60000 (43%)]	Loss: 0.234962
Train Epoch: 0 [32000/60000 (53%)]	Loss: 0.348309
Train Epoch: 0 [38400/60000 (64%)]	Loss: 0.191354
Train Epoch: 0 [44800/60000 (75%)]	Loss: 0.249863
Train Epoch: 0 [51200/60000 (85%)]	Loss: 0.202573
Train Epoch: 0 [57600/60000 (96%)]	Loss: 0.170296

Average loss: 0.1745, Accuracy: 56759/60000 (94.598%)

Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.088725
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.254969
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.149976
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.128624
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.285364
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.102782
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.336375
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.147386
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.167869

Average loss: 0.1335, Accuracy: 57509/60000 (

Train Epoch: 16 [19200/60000 (32%)]	Loss: 0.030013
Train Epoch: 16 [25600/60000 (43%)]	Loss: 0.053026
Train Epoch: 16 [32000/60000 (53%)]	Loss: 0.023321
Train Epoch: 16 [38400/60000 (64%)]	Loss: 0.091302
Train Epoch: 16 [44800/60000 (75%)]	Loss: 0.028209
Train Epoch: 16 [51200/60000 (85%)]	Loss: 0.047958
Train Epoch: 16 [57600/60000 (96%)]	Loss: 0.079661

Average loss: 0.0608, Accuracy: 58890/60000 (98.150%)

Train Epoch: 17 [6400/60000 (11%)]	Loss: 0.037211
Train Epoch: 17 [12800/60000 (21%)]	Loss: 0.018008
Train Epoch: 17 [19200/60000 (32%)]	Loss: 0.027271
Train Epoch: 17 [25600/60000 (43%)]	Loss: 0.028238
Train Epoch: 17 [32000/60000 (53%)]	Loss: 0.214740
Train Epoch: 17 [38400/60000 (64%)]	Loss: 0.025784
Train Epoch: 17 [44800/60000 (75%)]	Loss: 0.049863
Train Epoch: 17 [51200/60000 (85%)]	Loss: 0.001453
Train Epoch: 17 [57600/60000 (96%)]	Loss: 0.031715

Average loss: 0.0548, Accuracy: 59007/60000 (98.345%)

Train Epoch: 18 [6400/60000 (11%)]	Loss: 0.000828
Train Epoch: 18 [12800/

Train Epoch: 32 [19200/60000 (32%)]	Loss: 0.045142
Train Epoch: 32 [25600/60000 (43%)]	Loss: 0.110974
Train Epoch: 32 [32000/60000 (53%)]	Loss: 0.016292
Train Epoch: 32 [38400/60000 (64%)]	Loss: 0.028717
Train Epoch: 32 [44800/60000 (75%)]	Loss: 0.033552
Train Epoch: 32 [51200/60000 (85%)]	Loss: 0.000558
Train Epoch: 32 [57600/60000 (96%)]	Loss: 0.156964

Average loss: 0.0498, Accuracy: 59062/60000 (98.437%)

Train Epoch: 33 [6400/60000 (11%)]	Loss: 0.019424
Train Epoch: 33 [12800/60000 (21%)]	Loss: 0.200248
Train Epoch: 33 [19200/60000 (32%)]	Loss: 0.081500
Train Epoch: 33 [25600/60000 (43%)]	Loss: 0.008602
Train Epoch: 33 [32000/60000 (53%)]	Loss: 0.064084
Train Epoch: 33 [38400/60000 (64%)]	Loss: 0.129034
Train Epoch: 33 [44800/60000 (75%)]	Loss: 0.001996
Train Epoch: 33 [51200/60000 (85%)]	Loss: 0.021996
Train Epoch: 33 [57600/60000 (96%)]	Loss: 0.160991

Average loss: 0.0475, Accuracy: 59150/60000 (98.583%)

Train Epoch: 34 [6400/60000 (11%)]	Loss: 0.013164
Train Epoch: 34 [12800/

Train Epoch: 48 [19200/60000 (32%)]	Loss: 0.002668
Train Epoch: 48 [25600/60000 (43%)]	Loss: 0.056229
Train Epoch: 48 [32000/60000 (53%)]	Loss: 0.144359
Train Epoch: 48 [38400/60000 (64%)]	Loss: 0.000561
Train Epoch: 48 [44800/60000 (75%)]	Loss: 0.022782
Train Epoch: 48 [51200/60000 (85%)]	Loss: 0.017923
Train Epoch: 48 [57600/60000 (96%)]	Loss: 0.018786

Average loss: 0.0292, Accuracy: 59490/60000 (99.150%)

Train Epoch: 49 [6400/60000 (11%)]	Loss: 0.014171
Train Epoch: 49 [12800/60000 (21%)]	Loss: 0.002452
Train Epoch: 49 [19200/60000 (32%)]	Loss: 0.000408
Train Epoch: 49 [25600/60000 (43%)]	Loss: 0.060767
Train Epoch: 49 [32000/60000 (53%)]	Loss: 0.221991
Train Epoch: 49 [38400/60000 (64%)]	Loss: 0.195491
Train Epoch: 49 [44800/60000 (75%)]	Loss: 0.090293
Train Epoch: 49 [51200/60000 (85%)]	Loss: 0.132819
Train Epoch: 49 [57600/60000 (96%)]	Loss: 0.046763

Average loss: 0.0142, Accuracy: 59724/60000 (99.540%)

Train Epoch: 50 [6400/60000 (11%)]	Loss: 0.087692
Train Epoch: 50 [12800/

Train Epoch: 64 [19200/60000 (32%)]	Loss: 0.073163
Train Epoch: 64 [25600/60000 (43%)]	Loss: 0.014512
Train Epoch: 64 [32000/60000 (53%)]	Loss: 0.009101
Train Epoch: 64 [38400/60000 (64%)]	Loss: 0.020023
Train Epoch: 64 [44800/60000 (75%)]	Loss: 0.000291
Train Epoch: 64 [51200/60000 (85%)]	Loss: 0.009595
Train Epoch: 64 [57600/60000 (96%)]	Loss: 0.000244

Average loss: 0.0217, Accuracy: 59609/60000 (99.348%)



## Save or Load the Initial Model

In [10]:
# Save this model so that I dont have to train again in the future
torch.save(initial_model, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')

In [8]:
# Load the model from local file
initial_model = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')
initial_model.eval()

HNet(
  (flatten): Flatten()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

## Calculatee mse residual and normalize the result to values between 0 ~ 1

In [25]:
# Function used to calculate mse residual based on the wiki
def mseresidual(y, F):
    residual = y - F
    absolute = torch.abs(residual)
    residual = residual / torch.max(absolute)
    return residual

## Actual Code for gradient Boosting

In [10]:
#Main Function used to train and find optimal gamma for submodels
#input: intial model that's already trained
#M is number of submodels needed to be trained
def GradientBoosting(initial_model, M):
    gamma_exp = torch.ones([M], dtype = torch.float64) # used to hold the final optimized gamma
    models = [] # used to hold all the models
    residual_list = [] # used to hold the residual of each batch calculated
    for m in range(M):
        # Intialize submodels
        Hmodel = NHNet()
        Hcriterion = nn.MSELoss()
        if torch.cuda.is_available():
            Hmodel = Hmodel.cuda()
            gamma_exp = gamma_exp.cuda()
            Hcriterion = Hcriterion.cuda()
            
        # Start Training
        epoch = 40
        Hmodel.train()
        Hoptimizer = optim.Adam(Hmodel.parameters(), lr=0.003)
        for i in range(epoch):
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                # Create one-hot label target tensor
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                # Calculate F(x)
                Hoptimizer.zero_grad()
                output = initial_model(data)
                # Calculate the output from all the models
                for j in range(m):
                    model = models[j]
                    if torch.cuda.is_available():
                        output = output.cuda()
                        model = model.cuda()
                    output = output + gamma_exp[j] * model(data)
                    
                #Convert into Onehot label so that it would be able to calculate the residual
                target = target.view(-1,1)
                target_onehot.zero_()
                target_onehot.scatter_(1, target, 1)
                #Calculate Residual
                residual = mseresidual(target_onehot, output)
                houtput = Hmodel(data)
                residual = residual.type(torch.cuda.FloatTensor)
                houtput = houtput.type(torch.cuda.FloatTensor)
                #Calculate the loss
                loss = Hcriterion(houtput, residual)
                loss.backward(retain_graph = True)
                Hoptimizer.step()
                # Print out current Process
                if (batch_idx + 1)% 100 == 0 and i % 10 == 0:
                    print('Train Epoch: Model Number: {} {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        m+1,i, (batch_idx + 1) * len(data), len(train_loader.dataset),
                        100. * (batch_idx + 1) / len(train_loader), loss.item()))
        models.append(Hmodel)
        
        # Initialize a random gamma
        gamma = torch.rand(1, requires_grad=True, device="cuda")
        Goptimizer = optim.Adam([gamma], lr=0.01)
        Gcriterion = nn.MSELoss()
        # Start finding the best gamma
        for i in range(20):
            for batch_idx, (data, target) in enumerate(train_loader):
                data, target = Variable(data), Variable(target)
                nb_digits = 10
                target_onehot = torch.FloatTensor(data.shape[0], nb_digits)
                if torch.cuda.is_available():
                    data = data.cuda()
                    target = target.cuda()
                    target_onehot = target_onehot.cuda()
                    Hmodel = Hmodel.cuda()
                    gamma = gamma.cuda()
                    
                #Calculate the initial output
                Goptimizer.zero_grad()  
                output = initial_model(data)
                #Calculate the final output by combining all previous models
                for j in range(m):
                    model = models[j]
                    if torch.cuda.is_available():
                        model = model.cuda()
                        output = output.cuda()
                        gamma_temp = gamma_exp[j]
                        gamma_temp = gamma_temp.cuda()
                    output = output + gamma_temp * model(data)
                # Covert into one-hot label
                target = target.view(-1,1)
                target_onehot.zero_()
                target_onehot.scatter_(1, target, 1)
                # Get the currect model output
                temp = Hmodel(data)
                # Find the current ensemble model output
                predicted = output + gamma * temp
                # Calculate the loss
                loss = Gcriterion(predicted, target_onehot)
                loss.backward(retain_graph = True)
                # Optimize the gamma
                Goptimizer.step()
        gamma_exp[m] = gamma
    print(gamma_exp)
    return models, gamma_exp

## Start Gradient Boosting Training

In [26]:
num_of_models = 3
models, gamma_exp = GradientBoosting(initial_model, num_of_models)

Train Epoch: Model Number: 1 0 [6400/60000 (11%)]	Loss: 0.015326
Train Epoch: Model Number: 1 0 [12800/60000 (21%)]	Loss: 0.008539
Train Epoch: Model Number: 1 0 [19200/60000 (32%)]	Loss: 0.007648
Train Epoch: Model Number: 1 0 [25600/60000 (43%)]	Loss: 0.007757
Train Epoch: Model Number: 1 0 [32000/60000 (53%)]	Loss: 0.016929
Train Epoch: Model Number: 1 0 [38400/60000 (64%)]	Loss: 0.008677
Train Epoch: Model Number: 1 0 [44800/60000 (75%)]	Loss: 0.008257
Train Epoch: Model Number: 1 0 [51200/60000 (85%)]	Loss: 0.008708
Train Epoch: Model Number: 1 0 [57600/60000 (96%)]	Loss: 0.006351
Train Epoch: Model Number: 1 10 [6400/60000 (11%)]	Loss: 0.005690
Train Epoch: Model Number: 1 10 [12800/60000 (21%)]	Loss: 0.010076
Train Epoch: Model Number: 1 10 [19200/60000 (32%)]	Loss: 0.006353
Train Epoch: Model Number: 1 10 [25600/60000 (43%)]	Loss: 0.007768
Train Epoch: Model Number: 1 10 [32000/60000 (53%)]	Loss: 0.011292
Train Epoch: Model Number: 1 10 [38400/60000 (64%)]	Loss: 0.006347
Train 

## Save or Load submodels and optimized gamma

In [29]:
# Save all the models trained
for i in range(num_of_models):
    model = models[i]
    torch.save(model, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model' + str(i) + '.pth')

In [30]:
# Save the optimized gamma
torch.save(gamma_exp, 'C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp.txt')

In [83]:
# Load the models from the local files
num_of_models = 3
models = []
for x in range(num_of_models):
    globals()['model%s' % x] = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/model' + str(1) + '.pth')
    models.append(globals()['model%s' % x])
print(models)

[NHNet(
  (flatten): Flatten()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
), NHNet(
  (flatten): Flatten()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
), NHNet(
  (flatten): Flatten()
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)]


In [82]:
# Load the optimized gamma from the local files
gamma_exp = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/gamma_exp.txt')
print(gamma_exp)

tensor([1.0040, 0.9936, 1.0037], device='cuda:0', dtype=torch.float64,
       requires_grad=True)


## Ensemble Model Accuracy

In [27]:
#Find the accuracy of the ensemble model
initial_model = torch.load('C:/Users/cozyn/OneDrive/Desktop/Research/Adversarial-Machine-Learning/results/initial_model.pth')
initial_model.eval()

loss = 0
correct = 0
    
with torch.no_grad():
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data, volatile=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = initial_model(data)
        for i in range(num_of_models):
            model = models[i]
            if torch.cuda.is_available():
                model = model.cuda()
                output = output.cuda()
                gamma_temp = gamma_exp[i]
                gamma_temp = gamma_temp.cuda()
            output = output + gamma_temp * model(data)
        loss += F.cross_entropy(output, target, reduction='sum').item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
loss /= len(train_loader.dataset)  
print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.3f}%)\n'.format(
    loss, correct, len(train_loader.dataset),
    100. * correct / len(train_loader.dataset)))

C:\Users\cozyn\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # Remove the CWD from sys.path while we load stuff.



Average loss: 0.9301, Accuracy: 50175/60000 (83.625%)



## CW Attack on Intial Model, Testing Robust Accuracy

In [13]:
# Attack the initial model using CW attack
initial_model.eval()

correct = 0
total = 0

cw_attack = CW(initial_model, c=1)

for data, target in test_loader:

        images = cw_attack(data, target).cuda()
        outputs = initial_model(images)
        
        _, predicted = torch.max(outputs.data, 1)

        total += target.size(0)
        correct += (predicted == target.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 28.78 %


## Creating Ensemble CW Attack

In [14]:
# Create ensemble CW attack
class Attack_ensemble(object):
    r"""
    Base class for all attacks.
    .. note::
        It automatically set device to the device where given model is.
        It temporarily changes the original model's training mode to `test`
        by `.eval()` only during an attack process.
    """
    def __init__(self, name, model, models, gamma):
        r"""
        Initializes internal attack state.
        Arguments:
            name (str) : name of an attack.
            model (torch.nn.Module): model to attack.
        """

        self.attack = name
        self.model = model
        self.models = models
        self.gamma = gamma
        self.model_name = str(model).split("(")[0]

        self.training = model.training
        self.device = next(model.parameters()).device
        
        self._targeted = 1
        self._attack_mode = 'original'
        self._return_type = 'float'

    def forward(self, *input):
        r"""
        It defines the computation performed at every call.
        Should be overridden by all subclasses.
        """
        raise NotImplementedError
        
    def set_attack_mode(self, mode):
        r"""
        Set the attack mode.
  
        Arguments:
            mode (str) : 'original' (DEFAULT)
                         'targeted' - Use input labels as targeted labels.
                         'least_likely' - Use least likely labels as targeted labels.
        """
        if self._attack_mode is 'only_original':
            raise ValueError("Changing attack mode is not supported in this attack method.")
            
        if mode=="original":
            self._attack_mode = "original"
            self._targeted = 1
            self._transform_label = self._get_label
        elif mode=="targeted":
            self._attack_mode = "targeted"
            self._targeted = -1
            self._transform_label = self._get_label
        elif mode=="least_likely":
            self._attack_mode = "least_likely"
            self._targeted = -1
            self._transform_label = self._get_least_likely_label
        else:
            raise ValueError(mode + " is not a valid mode. [Options : original, targeted, least_likely]")
            
    def set_return_type(self, type):
        r"""
        Set the return type of adversarial images: `int` or `float`.
        Arguments:
            type (str) : 'float' or 'int'. (DEFAULT : 'float')
        """
        if type == 'float':
            self._return_type = 'float'
        elif type == 'int':
            self._return_type = 'int'
        else:
            raise ValueError(type + " is not a valid type. [Options : float, int]")

    def save(self, save_path, data_loader, verbose=True):
        r"""
        Save adversarial images as torch.tensor from given torch.utils.data.DataLoader.
        Arguments:
            save_path (str) : save_path.
            data_loader (torch.utils.data.DataLoader) : data loader.
            verbose (bool) : True for displaying detailed information. (DEFAULT : True)
        """
        self.model.eval()

        image_list = []
        label_list = []

        correct = 0
        total = 0

        total_batch = len(data_loader)

        for step, (images, labels) in enumerate(data_loader):
            adv_images = self.__call__(images, labels)

            image_list.append(adv_images.cpu())
            label_list.append(labels.cpu())

            if self._return_type == 'int':
                adv_images = adv_images.float()/255

            if verbose:
                outputs = self.model(adv_images)
                for i in range(len(self.models)):
                    sub_model = self.models[i]
                    outputs = outputs + self.gamma[i] * sub_model(adv_images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels.to(self.device)).sum()

                acc = 100 * float(correct) / total
                print('- Save Progress : %2.2f %% / Accuracy : %2.2f %%' % ((step+1)/total_batch*100, acc), end='\r')

        x = torch.cat(image_list, 0)
        y = torch.cat(label_list, 0)
        torch.save((x, y), save_path)
        print('\n- Save Complete!')

        self._switch_model()
        
    def _transform_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        """
        return labels
        
    def _get_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        Return input labels.
        """
        return labels
    
    def _get_least_likely_label(self, images, labels):
        r"""
        Function for changing the attack mode.
        Return least likely labels.
        """
        outputs = self.model(images)
        for i in range(len(self.models)):
            sub_model = self.models[i]
            outputs = outputs + self.gamma[i] * sub_model(images)
        _, labels = torch.min(outputs.data, 1)
        labels = labels.detach_()
        return labels
    
    def _to_uint(self, images):
        r"""
        Function for changing the return type.
        Return images as int.
        """
        return (images*255).type(torch.uint8)

    def _switch_model(self):
        r"""
        Function for changing the training mode of the model.
        """
        if self.training:
            self.model.train()
            for i in range(len(self.models)):
                self.models[i].train()
        else:
            self.model.eval()
            for i in range(len(self.models)):
                self.models[i].eval()

    def __str__(self):
        info = self.__dict__.copy()
        
        del_keys = ['model', 'attack']
        
        for key in info.keys():
            if key[0] == "_" :
                del_keys.append(key)
                
        for key in del_keys:
            del info[key]
        
        info['attack_mode'] = self._attack_mode
        if info['attack_mode'] == 'only_original' :
            info['attack_mode'] = 'original'
            
        info['return_type'] = self._return_type
        
        return self.attack + "(" + ', '.join('{}={}'.format(key, val) for key, val in info.items()) + ")"

    def __call__(self, *input, **kwargs):
        self.model.eval()
        for i in range(len(self.models)):
            self.models[i].eval()
        images = self.forward(*input, **kwargs)
        self._switch_model()

        if self._return_type == 'int':
            images = self._to_uint(images)

        return images

In [15]:
# Create Ensemble CW Attack
import warnings

class CW_Ensemble(Attack_ensemble):
    r"""
    CW in the paper 'Towards Evaluating the Robustness of Neural Networks'
    [https://arxiv.org/abs/1608.04644]
    Distance Measure : L2
        
    Arguments:
        model (nn.Module): model to attack.
        c (float): c in the paper. parameter for box-constraint. (DEFALUT : 1e-4)    
            :math:`minimize \Vert\frac{1}{2}(tanh(w)+1)-x\Vert^2_2+c\cdot f(\frac{1}{2}(tanh(w)+1))`    
        kappa (float): kappa (also written as 'confidence') in the paper. (DEFALUT : 0)
            :math:`f(x')=max(max\{Z(x')_i:i\neq t\} -Z(x')_t, - \kappa)`
        steps (int): number of steps. (DEFALUT : 1000)
        lr (float): learning rate of the Adam optimizer. (DEFALUT : 0.01)
        
    .. warning:: With default c, you can't easily get adversarial images. Set higher c like 1.
    
    Shape:
        - images: :math:`(N, C, H, W)` where `N = number of batches`, `C = number of channels`,        `H = height` and `W = width`. It must have a range [0, 1].
        - labels: :math:`(N)` where each value :math:`y_i` is :math:`0 \leq y_i \leq` `number of labels`.
        - output: :math:`(N, C, H, W)`.
          
    Examples::
        >>> attack = torchattacks.CW(model, targeted=False, c=1e-4, kappa=0, steps=1000, lr=0.01)
        >>> adv_images = attack(images, labels)
        
    .. note:: NOT IMPLEMENTED methods in the paper due to time consuming.
    
        (1) Binary search for c.
        
        (2) Choosing best L2 adversaries.
    """
    def __init__(self, model, models, gamma, c=1e-4, kappa=0, steps=1000, lr=0.01):
        super(CW_Ensemble, self).__init__("CW", model, models, gamma)
        self.c = c
        self.kappa = kappa
        self.steps = steps
        self.lr = lr

    def forward(self, images, labels):
        r"""
        Overridden.
        """
        images = images.to(self.device)
        labels = labels.to(self.device)
        labels = self._transform_label(images, labels)

        # f-function in the paper
        def f(x):
            outputs = self.model(x)
            for i in range(len(self.models)):
                sub_model = self.models[i]
                outputs = outputs + self.gamma[i] * sub_model(x)
            one_hot_labels = torch.eye(len(outputs[0]))[labels].to(self.device)

            i, _ = torch.max((1-one_hot_labels)*outputs, dim=1)
            j = torch.masked_select(outputs, one_hot_labels.bool())

            return torch.clamp(self._targeted*(j-i), min=-self.kappa)

        w = torch.zeros_like(images).to(self.device)
        w.detach_()
        w.requires_grad = True

        optimizer = optim.Adam([w], lr=self.lr)
        prev = 1e10

        for step in range(self.steps):

            a = 1/2*(nn.Tanh()(w) + 1)

            loss1 = nn.MSELoss(reduction='sum')(a, images)
            loss2 = torch.sum(self.c*f(a))

            cost = loss1 + loss2

            optimizer.zero_grad()
            cost.backward(retain_graph=True)
            optimizer.step()

            # Early Stop when loss does not converge.
            if step % (self.steps//10) == 0:
                if cost > prev:
                    warnings.warn("Early stopped because the loss is not converged.")
                    return (1/2*(nn.Tanh()(w) + 1)).detach()
                prev = cost

            # print('- CW Attack Progress : %2.2f %%        ' %((step+1)/self.steps*100), end='\r')

        adv_images = (1/2*(nn.Tanh()(w) + 1)).detach()

        return adv_images

## CW Attack Ensemble Model, Testing Robust Accuracy

In [28]:
#Attack the ensemble model
initial_model.eval()
for i in range(num_of_models):
    models[i].eval()

correct = 0
total = 0

cw_attack = CW_Ensemble(model = initial_model,models = models,gamma = gamma_exp, c=1)

for data, target in test_loader:

        images = cw_attack(data, target).cuda()
        outputs = initial_model(images)
        for i in range(num_of_models):
            sub_model = models[i]
            outputs = outputs + gamma_exp[i] * sub_model(images)
        _, predicted = torch.max(outputs.data, 1)

        total += target.size(0)
        correct += (predicted == target.cuda()).sum()
    
print('Robust accuracy: %.2f %%' % (100 * float(correct) / total))

Robust accuracy: 5.33 %
